<a href="https://colab.research.google.com/github/paparon96/Sustainability/blob/main/Carbon_price_prediction/Notebooks/ols_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
from statsmodels.regression.linear_model import OLS
from statsmodels.tsa.stattools import adfuller
import statsmodels.formula.api as smf
import statsmodels.api as sm
import pandas as pd
import copy
import matplotlib.pyplot as plt


import os
os.chdir('/content/drive/My Drive/Carbon_price_prediction/Workspace/Data')

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [30]:
def diff_for_stationarity(df_input,cols,threshold,verbose=False):
    
    """Checks the series for stationarity then difference it until it is stationary
    (with the given threshold)"""
    
    df = copy.deepcopy(df_input)
    
    
    # Get length of DF
    n = len(df)
    
    # Loop over columns of interest
    for col in cols:
        stationarity = False
        series = list(df[col].astype(float))
        linear_diff = False
                
        try:
        
            counter = 0



            while not stationarity:

                #print(counter)

                # Run ADF test
                adf_test = adfuller(series)
                pval = adf_test[1]

                if pval < threshold:
                    stationarity = True

                else:
                    # Difference the series
                    series = [np.log(series[i]/series[i-1]) for i in range(1,n-counter)]
                    #series = [series[i]-series[i-1] for i in range(1,n-counter)]

                    counter += 1
                    
        except:
            
            counter = 0
            stationarity = False
            series = list(df[col].astype(float))
            linear_diff = True

            while not stationarity:

                #print(counter)

                # Run ADF test
                adf_test = adfuller(series)
                pval = adf_test[1]

                if pval < threshold:
                    stationarity = True

                elif counter>1:
                    stationarity = True
                    series = list(df[col].astype(float))
                else:
                    # Difference the series
                    series = [series[i]-series[i-1] for i in range(1,n-counter)]

                    counter += 1
        
        # Create final series and assign it back to the dataframe
        final_series = [None]*counter + series
        df[col] = final_series
        
        if not linear_diff:
            type_diff = "log difference"
        else:
            type_diff = "linear difference"
        if verbose:
            print("Variable: {} is {}-order integrated ({})!".format(col,counter,type_diff))
        
        
    return df

In [16]:
predictors = pd.read_csv("merged_articles.csv")

In [17]:
predictors

,article,simple_target,log_target,categorical_target,date,source,binary_target,carbon_price,oil_price,gas_price,stock_market_index_level,energy_price,coal_price
0,world economic forum two consecutive years res...,-0.69,-0.119623,0,2017-01-03,0,0,5.43,55.47,49.45,365.71,NaN,82.50
1,eu market euas fall 6 months energy weighs auc...,-0.69,-0.119623,0,2017-01-03,1,0,5.43,55.47,49.45,365.71,NaN,82.50
2,working towards completion tycoon bernard tapi...,-0.69,-0.119623,0,2017-01-03,0,0,5.43,55.47,49.45,365.71,NaN,82.50
3,israeli pm probed cash claims french carbon fr...,-0.69,-0.119623,0,2017-01-03,1,0,5.43,55.47,49.45,365.71,NaN,82.50
4,ten million degrees russian science foundation...,-0.69,-0.119623,0,2017-01-03,0,0,5.43,55.47,49.45,365.71,NaN,82.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5927,upward trend compared suez canal bloceage shel...,-0.63,-0.014294,0,2021-04-13,0,0,43.76,63.67,48.60,435.75,2568.57,68.85
5928,shipbrokers charting course trade carbon shipb...,-0.63,-0.014294,0,2021-04-13,1,0,43.76,63.67,48.60,435.75,2568.57,68.85
5929,eu market euas slip back €44 profit taking hit...,-0.63,-0.014294,0,2021-04-13,1,0,43.76,63.67,48.60,435.75,2568.57,68.85
5930,use multiple subscriptions unevenly accessible...,-0.63,-0.014294,0,2021-04-13,0,0,43.76,63.67,48.60,435.75,2568.57,68.85


In [ ]:
df = predictors

In [ ]:
signal1 = pd.read_csv("cat2_predictions_lstm.csv")
signal2 = pd.read_csv("num_predictions_regression_cnn.csv")
signal3 = pd.read_csv("num_predictions_lstm_diff.csv")

In [ ]:
df = pd.merge(predictors, signal1, on="date")
df = pd.merge(df, signal2, on="date")
df = pd.merge(df, signal3, on="date")

In [ ]:
df = df.groupby('date').mean()

In [ ]:
df.head()

,simple_target,log_target,categorical_target,source,binary_target,carbon_price,oil_price,gas_price,stock_market_index_level,energy_price,coal_price,pred_x,target_x,pred_y,target_y,text_feature,pred,target
date,,,,,,,,,,,,,,,,,,
2017-09-27,-0.05,-0.007189,1.0,0.461538,0.0,6.93,57.90,46.32,385.62,1020.29,91.50,0.544859,0.0,-0.010888,-0.007189,0.0,-0.043365,0.224929
2017-09-28,0.03,0.004320,2.0,0.600000,1.0,6.96,57.41,46.45,386.36,1022.39,91.45,0.533136,1.0,-0.010404,0.004320,0.0,-0.137377,0.143931
2017-09-29,0.74,0.101041,3.0,0.666667,1.0,7.70,57.54,47.32,388.16,1019.04,91.40,0.519479,1.0,-0.009336,0.101041,0.0,-0.057870,-0.481756
2017-10-02,-0.76,-0.103919,0.0,0.600000,0.0,6.94,56.12,48.22,390.13,1024.34,90.65,0.557878,0.0,-0.011871,-0.103919,0.0,0.171293,0.668878
2017-10-03,0.06,0.008608,2.0,0.200000,1.0,7.00,56.00,49.24,390.72,1026.63,91.80,0.532884,1.0,-0.009511,0.008608,0.0,-0.070146,0.240814


In [ ]:
df.dropna(inplace=True)

In [ ]:
stacioner_threshold = 0.01
arno_cols = df.columns
current_df = df

# Check for stationarity
df_stationer = diff_for_stationarity(df_input=current_df,cols=arno_cols,threshold=stacioner_threshold)

Variable: simple_target is 0-order integrated (log difference)!
Variable: log_target is 0-order integrated (log difference)!
Variable: categorical_target is 0-order integrated (log difference)!
Variable: source is 0-order integrated (log difference)!
Variable: binary_target is 0-order integrated (log difference)!
Variable: carbon_price is 1-order integrated (linear difference)!
Variable: oil_price is 1-order integrated (linear difference)!
Variable: gas_price is 1-order integrated (linear difference)!
Variable: stock_market_index_level is 1-order integrated (linear difference)!
Variable: energy_price is 1-order integrated (linear difference)!
Variable: coal_price is 1-order integrated (linear difference)!
Variable: pred_x is 0-order integrated (log difference)!
Variable: target_x is 0-order integrated (log difference)!
Variable: pred_y is 0-order integrated (log difference)!
Variable: target_y is 0-order integrated (log difference)!
Variable: text_feature is 0-order integrated (log dif

In [ ]:
df_stationer.dropna(inplace=True)

In [ ]:
#y = df.carbon_price[1:]
y = df_stationer.carbon_price

In [ ]:
#x = df.loc[:,['oil_price', 'gas_price', 'stock_market_index_level', 'energy_price', 'coal_price', 'pred']][:-1]
x = df_stationer.loc[:,['oil_price', 'gas_price', 'stock_market_index_level', 'energy_price', 'coal_price','pred',# 'pred_x', 'pred_y', 'text_feature'
]]

In [ ]:
x = sm.add_constant(x)

In [ ]:
model = sm.OLS(y,x)
results = model.fit()

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           carbon_price   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     32.63
Date:                Wed, 27 Oct 2021   Prob (F-statistic):           2.06e-35
Time:                        13:38:42   Log-Likelihood:                -797.91
No. Observations:                 844   AIC:                             1610.
Df Residuals:                     837   BIC:                             1643.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [ ]:
#x = df.loc[:,['oil_price', 'gas_price', 'stock_market_index_level', 'energy_price', 'coal_price', 'pred']][:-1]
x = df_stationer.loc[:,['oil_price', 'gas_price', 'stock_market_index_level', 'energy_price', 'coal_price']]

In [ ]:
x = sm.add_constant(x)

In [ ]:
model = sm.OLS(y,x)
results = model.fit()

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           carbon_price   R-squared:                       0.188
Model:                            OLS   Adj. R-squared:                  0.184
Method:                 Least Squares   F-statistic:                     39.23
Date:                Tue, 26 Oct 2021   Prob (F-statistic):           2.72e-36
Time:                        13:14:50   Log-Likelihood:                -802.61
No. Observations:                 851   AIC:                             1617.
Df Residuals:                     845   BIC:                             1646.
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
const                   

In [ ]:
difference = results.fittedvalues - y

In [ ]:
difference.to_csv("difference.csv", index=False)

In [ ]:
data = df_stationer.loc[:,['carbon_price', 'oil_price', 'gas_price',
       'stock_market_index_level', 'energy_price', 'coal_price','pred', 'pred_x', 'pred_y', 'text_feature']]
for lag in range(1, 4):
        lag_data = data.shift( lag )
        lag_data.columns = [ str( col ) + f'_lag{ lag }' for col in lag_data.columns ]
        data = pd.concat( [ data, lag_data ], axis = 1 )

In [ ]:
data.dropna(inplace=True)

In [ ]:
y = data.carbon_price

In [ ]:
x = data.drop(["carbon_price"], axis=1)

In [ ]:
x = sm.add_constant(x)

In [ ]:
model = sm.OLS(y,x)
results = model.fit()

In [ ]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           carbon_price   R-squared:                       0.266
Model:                            OLS   Adj. R-squared:                  0.200
Method:                 Least Squares   F-statistic:                     4.039
Date:                Wed, 27 Oct 2021   Prob (F-statistic):           5.54e-22
Time:                        13:37:34   Log-Likelihood:                -752.37
No. Observations:                 838   AIC:                             1645.
Df Residuals:                     768   BIC:                             1976.
Df Model:                          69                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------

In [ ]:
difference = results.fittedvalues - y

In [ ]:
difference.head()

date
2017-09-27    0.224929
2017-09-28    0.143931
2017-09-29   -0.481756
2017-10-02    0.668878
2017-10-03    0.240814
dtype: float64

In [ ]:
difference.to_csv("difference.csv")

In [ ]:

def forward_selected( df, response, lags ):
    data = df
    for lag in range( 1, lags + 1 ):
        lag_data = df.shift( lag )
        lag_data.columns = [ str( col ) + f'_lag{ lag }' for col in lag_data.columns ]
        data = pd.concat( [ data, lag_data ], axis = 1 )
    
    # Add constant
    # data = sm.add_constant( data )
    remaining = set( data.columns )

    remaining.difference_update(df.columns)
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(sorted(selected + [candidate])))
            score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(sorted(selected)))
    model = smf.ols(formula, data).fit()
    return model

In [ ]:
lags = 5
response = 'carbon_price'
model = forward_selected( df_stationer, response, lags )
print( model.summary() )

# Automated regressions

In [63]:
def simple_OLS(signal_name, verbose=True):
    stacioner_threshold = 0.01

    data = pd.read_csv("merged_articles.csv")
    data = data.loc[:,['date', 'carbon_price', 'oil_price', 'gas_price','stock_market_index_level', 'energy_price', 'coal_price']]
    signal = pd.read_csv(f'https://raw.githubusercontent.com/paparon96/Sustainability/main/Carbon_price_prediction/Data/signals/{signal_name}.csv', index_col=0)
    df = pd.merge(data, signal, on="date", how='left')
    df = df.groupby('date').mean()
    df.dropna(inplace=True)

    # Check for stationarity
    arno_cols = df.columns
    df_stationer = diff_for_stationarity(df_input=df, cols=arno_cols, threshold=stacioner_threshold, verbose=False)
    df_stationer.dropna(inplace=True)

    y = df_stationer.carbon_price
    x = df_stationer.drop(["carbon_price"], axis=1)
    x = sm.add_constant(x)

    model = sm.OLS(y,x)
    results = model.fit()

    print('----- Signal: {} ----------------'.format(signal_name))
    if verbose:
        print(results.summary())
    else:
        print(results.rsquared)

In [48]:
simple_OLS('binary_prob')

----- Signal: binary_prob ----------------
                            OLS Regression Results                            
Dep. Variable:           carbon_price   R-squared:                       0.190
Model:                            OLS   Adj. R-squared:                  0.185
Method:                 Least Squares   F-statistic:                     33.06
Date:                Wed, 10 Nov 2021   Prob (F-statistic):           6.85e-36
Time:                        16:31:13   Log-Likelihood:                -801.62
No. Observations:                 851   AIC:                             1617.
Df Residuals:                     844   BIC:                             1650.
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------

In [64]:
def lagged_OLS(signal_name, verbose=True):
    stacioner_threshold = 0.01

    data = pd.read_csv("merged_articles.csv")
    data = data.loc[:,['date', 'carbon_price', 'oil_price', 'gas_price','stock_market_index_level', 'energy_price', 'coal_price']]
    signal = pd.read_csv(f'https://raw.githubusercontent.com/paparon96/Sustainability/main/Carbon_price_prediction/Data/signals/{signal_name}.csv', index_col=0)
    df = pd.merge(data, signal, on="date", how='left')
    df = df.groupby('date').mean()
    df.dropna(inplace=True)

    # Check for stationarity
    arno_cols = df.columns
    df_stationer = diff_for_stationarity(df_input=df, cols=arno_cols, threshold=stacioner_threshold, verbose=False)
    df_stationer.dropna(inplace=True)

    for lag in range(1, 4):
        lag_data = df_stationer.shift(lag)
        lag_data.columns = [str(col) + f'_lag{lag}' for col in lag_data.columns]
        df_stationer = pd.concat([df_stationer, lag_data ], axis=1)

    df_stationer.dropna(inplace=True)

    y = df_stationer.carbon_price
    x = df_stationer.drop(["carbon_price"], axis=1)
    x = sm.add_constant(x)

    model = sm.OLS(y,x)
    results = model.fit()

    print('----- Signal: {} ----------------'.format(signal_name))
    if verbose:
        print(results.summary())
    else:
        print(results.rsquared)

In [65]:
lagged_OLS('binary_prob')

----- Signal: binary_prob ----------------
                            OLS Regression Results                            
Dep. Variable:           carbon_price   R-squared:                       0.255
Model:                            OLS   Adj. R-squared:                  0.210
Method:                 Least Squares   F-statistic:                     5.677
Date:                Wed, 10 Nov 2021   Prob (F-statistic):           1.67e-27
Time:                        16:40:44   Log-Likelihood:                -762.76
No. Observations:                 845   AIC:                             1624.
Df Residuals:                     796   BIC:                             1856.
Df Model:                          48                                         
Covariance Type:            nonrobust                                         
                                              coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------

In [66]:
signal_names = ['binary_prob', 'log_difference', 'cp_news_signal', 'residuals', 'text_feature']

In [67]:
for signal in signal_names:
    simple_OLS(signal, verbose=False)

----- Signal: binary_prob ----------------
0.1902876583258446
----- Signal: log_difference ----------------
0.1884478934908661
----- Signal: cp_news_signal ----------------
0.19745410966196297
----- Signal: residuals ----------------
0.18954275475015236
----- Signal: text_feature ----------------
0.18869051576605056
